<a href="https://colab.research.google.com/github/czasoprzestrzenny/public_repo/blob/master/new_data_source_coronavirus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#author czasoprzestrzenny 15.03.2020
#data source : https://data.humdata.org/dataset/5dff64bc-a671-48da-aa87-2ca40d7abf02
#Johns Hopkins School of Public Health

In [0]:
pip install folium

In [0]:
pip install chart_studio

     |████████████████████████████████| 81kB 2.7MB/s 


In [0]:
import warnings
warnings.filterwarnings("ignore")
import folium as folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins
import datetime 
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots


In [0]:
url_confirmed = """https://data.humdata.org/hxlproxy/data/download/time_series-ncov-Confirmed.csv?dest=data_edit&filter01=explode&explode-header-att01=date&explode-value-att01=value&filter02=rename&rename-oldtag02=%23affected%2Bdate&rename-newtag02=%23date&rename-header02=Date&filter03=rename&rename-oldtag03=%23affected%2Bvalue&rename-newtag03=%23affected%2Binfected%2Bvalue%2Bnum&rename-header03=Value&filter04=clean&clean-date-tags04=%23date&filter05=sort&sort-tags05=%23date&sort-reverse05=on&filter06=sort&sort-tags06=%23country%2Bname%2C%23adm1%2Bname&tagger-match-all=on&tagger-default-tag=%23affected%2Blabel&tagger-01-header=province%2Fstate&tagger-01-tag=%23adm1%2Bname&tagger-02-header=country%2Fregion&tagger-02-tag=%23country%2Bname&tagger-03-header=lat&tagger-03-tag=%23geo%2Blat&tagger-04-header=long&tagger-04-tag=%23geo%2Blon&header-row=1&url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Confirmed.csv"""
read_confirmed=pd.read_csv(url_confirmed)
url_recovered = """https://data.humdata.org/hxlproxy/data/download/time_series-ncov-Recovered.csv?dest=data_edit&filter01=explode&explode-header-att01=date&explode-value-att01=value&filter02=rename&rename-oldtag02=%23affected%2Bdate&rename-newtag02=%23date&rename-header02=Date&filter03=rename&rename-oldtag03=%23affected%2Bvalue&rename-newtag03=%23affected%2Brecovered%2Bvalue%2Bnum&rename-header03=Value&filter04=clean&clean-date-tags04=%23date&filter05=sort&sort-tags05=%23date&sort-reverse05=on&filter06=sort&sort-tags06=%23country%2Bname%2C%23adm1%2Bname&tagger-match-all=on&tagger-default-tag=%23affected%2Blabel&tagger-01-header=province%2Fstate&tagger-01-tag=%23adm1%2Bname&tagger-02-header=country%2Fregion&tagger-02-tag=%23country%2Bname&tagger-03-header=lat&tagger-03-tag=%23geo%2Blat&tagger-04-header=long&tagger-04-tag=%23geo%2Blon&header-row=1&url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Recovered.csv"""
read_recovered=pd.read_csv(url_recovered)
url_deaths = """https://data.humdata.org/hxlproxy/data/download/time_series-ncov-Deaths.csv?dest=data_edit&filter01=explode&explode-header-att01=date&explode-value-att01=value&filter02=rename&rename-oldtag02=%23affected%2Bdate&rename-newtag02=%23date&rename-header02=Date&filter03=rename&rename-oldtag03=%23affected%2Bvalue&rename-newtag03=%23affected%2Bkilled%2Bvalue%2Bnum&rename-header03=Value&filter04=clean&clean-date-tags04=%23date&filter05=sort&sort-tags05=%23date&sort-reverse05=on&filter06=sort&sort-tags06=%23country%2Bname%2C%23adm1%2Bname&tagger-match-all=on&tagger-default-tag=%23affected%2Blabel&tagger-01-header=province%2Fstate&tagger-01-tag=%23adm1%2Bname&tagger-02-header=country%2Fregion&tagger-02-tag=%23country%2Bname&tagger-03-header=lat&tagger-03-tag=%23geo%2Blat&tagger-04-header=long&tagger-04-tag=%23geo%2Blon&header-row=1&url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Deaths.csv"""
read_deaths=pd.read_csv(url_deaths)

In [0]:
data_c = read_confirmed.loc[1:]
data_r = read_recovered.loc[1:]
data_d = read_deaths.loc[1:]

data_c.rename(columns={'Value':'total_cases'}, inplace=True)
data_r.rename(columns={'Value':'total_recovered'}, inplace=True)
data_d.rename(columns={'Value':'total_deaths'}, inplace=True)

In [0]:
def prepare_to_plot(locations,frame,column_on_x, column_on_y, chart_title, xaxis_title,  yaxis_title, chart_type = "Bar"):
  """prepare data to ploting"""
  data = []

  for i, item in enumerate(locations):
        
        next_loc = pd.DataFrame(frame[frame['Province/State'] == item])
        #print (item)

        if chart_type =="Scatter":
          trace = go.Scatter(
          x=next_loc[column_on_y],
          y=next_loc[column_on_x],
          name=str(item))
        elif chart_type =="Bar":
          trace = go.Bar(
          x=next_loc[column_on_y],
          y=next_loc[column_on_x],
          name=str(item))

        layout = dict(title= chart_title + ". Generated : " + str('{:%Y-%m-%d}'.format(datetime.datetime.now())),
                      xaxis=dict(title= column_on_y),
                      yaxis=dict(title='col'),
                      xaxis_title= xaxis_title,
                      yaxis_title= yaxis_title,
                      )

        data.append(trace)

  data = dict(data=data, layout=layout)
  return data

In [0]:
data_c_r= data_c.merge(data_r, 'inner', on=['Country/Region', 'Date', 'Province/State'])
data_final = data_c_r.merge(data_d, 'inner', on=['Country/Region', 'Date', 'Province/State'])

In [0]:
data = data_final[['Province/State', 'Country/Region', 'Lat_x', 'Long_x', 'Date', 'total_cases', 'total_recovered', 'total_deaths']]
data['Province/State'] = data['Province/State'].where(pd.notnull, data['Country/Region'])
data.sort_values(['Province/State','Country/Region', 'Date'], ascending=False, inplace= True)
data["total_cases"] = data["total_cases"].apply(pd.to_numeric)
data[["Lat_x", "Long_x", "total_cases", "total_deaths"]] = data[["Lat_x", "Long_x", "total_cases","total_deaths"]].apply(pd.to_numeric)
data["Lat_x"] = data["Lat_x"].apply(pd.to_numeric)
data['Province/State']= data['Province/State'].astype(str) 
data['yesterday_cases'] = data.groupby(['Province/State','Country/Region'])['total_cases'].shift(-1)
data['yesterday_deaths'] = data.groupby(['Province/State','Country/Region'])['total_deaths'].shift(-1)
data['new_cases'] = data['total_cases'] - data['yesterday_cases']
data['new_deaths'] = data['total_deaths'] - data['yesterday_deaths']
data[["new_cases", "total_deaths", "total_recovered"]] = data[["new_cases", "total_deaths","total_recovered"]].apply(pd.to_numeric)
data['PercentOfNew'] =  (data.total_cases / (data.total_cases - data.new_cases)) -1
data['FatalityRate'] =  data.total_deaths / data.total_cases
data['lat_long'] = list(zip(data.Lat_x, data.Long_x))
data['Country/Region']= data['Country/Region'].apply(lambda x:x.upper())
data['Province/State']= data['Province/State'].apply(lambda x:x.upper())
#[data[col].apply(lambda x:x.upper()) for col in ['Country/Region', 'Province/State']]
data['Date'] = pd.to_datetime(data['Date'])
#data.head(5)

In [0]:
final_frame_map = data.drop_duplicates(subset=['Province/State','Country/Region'], keep= 'first')

In [0]:
lat_long = final_frame_map.lat_long.values.tolist()
total_cases = final_frame_map.total_cases.values.tolist()
location = final_frame_map['Province/State'].unique().tolist()

In [0]:
def color_change(c):
    """ convert a points to colors"""
    if(c < 100):
        return('green')
    elif(100 <= c < 1000):
        return('orange')
    else:
        return('red')

In [0]:
map = folium.Map()

for point in range(0, len(lat_long)):

    folium.CircleMarker(lat_long[point], 
                        radius = 9,
                        #popup=str(total_cases[point]), 
                        tooltip = str(total_cases[point]),
                        fill_color = color_change(total_cases[point]),
                        fill_opacity = 0.6               
                        ).add_to(map)

plugins.MarkerCluster(lat_long).add_to(map)

#plugins.TimestampedGeoJson({
#    'type': 'FeatureCollection',
#}, period='PT1M', add_last_point=True).add_to(map)
map

In [0]:
## choose data to further analysis

In [0]:
location_eu = [loc for loc in location if loc in ['UNITED KINGDOM', 'ITALY', 'SPAIN', 'GERMANY', 'FRANCE', 'POLAND', \
                                                  'SLOVAKIA', 'UKRAINE','CZECH REPUBLIC']]
final_frame_to_chart = data[data['Date']> pd.Timestamp(2020, 3, 1)]
final_frame_to_chart['is_country'] = final_frame_to_chart['Province/State']==final_frame_to_chart['Country/Region']
final_frame_to_chart = final_frame_to_chart[['Province/State','Country/Region', 'Date', 'total_cases', 'total_recovered', 'total_deaths', 'new_cases', \
                                             'new_deaths', 'PercentOfNew', 'FatalityRate', 'is_country']]

In [0]:
pre_plot = prepare_to_plot(locations = location_eu, frame = final_frame_to_chart, 
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_CASES for european countries", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [0]:
pre_plot = prepare_to_plot(locations = location_eu, frame = final_frame_to_chart, 
                           column_on_x = 'total_deaths', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_DEATHS for european countries", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [0]:
pre_plot = prepare_to_plot(locations = location_eu, frame = final_frame_to_chart, 
                           column_on_x = 'new_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / NUMBER OF NEW_CASES (day by day) for european countries", 
                           xaxis_title ="Date",  yaxis_title = "% of new cases", chart_type= 'Bar')
 
iplot(pre_plot)

In [0]:
# the biggest number of total deaths

In [0]:
frame_new = final_frame_to_chart[final_frame_to_chart['is_country']==True]
frame_new = frame_new.sort_values(['Date', 'total_deaths'], ascending=False)
location_new = frame_new['Province/State'].head(5).unique().tolist()
frame_new = frame_new[frame_new['Date']> pd.Timestamp(2020, 3, 7)]
frame_new.head(5)

,Province/State,Country/Region,Date,total_cases,total_recovered,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
8662,ITALY,ITALY,2020-03-22,59138,7024,5476,5560.0,651.0,0.103774,0.092597,True
12749,SPAIN,SPAIN,2020-03-22,28768,2575,1772,3394.0,397.0,0.133759,0.061596,True
8418,IRAN,IRAN,2020-03-22,21638,7931,1685,1028.0,129.0,0.049879,0.077872,True
6832,FRANCE,FRANCE,2020-03-22,16018,2200,674,1736.0,112.0,0.121552,0.042078,True
14213,UNITED KINGDOM,UNITED KINGDOM,2020-03-22,5683,65,281,665.0,48.0,0.132523,0.049446,True


In [0]:
pre_plot = prepare_to_plot(locations = location_new, frame = frame_new, 
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_CASES for location with highest value of total deaths", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [0]:
##the highest value of Death Rate

In [0]:
frame_new.sort_values(['Date','Province/State'], ascending=False, inplace= True)
final_frame_new_deaths = frame_new.drop_duplicates(subset=['Date','Province/State'], keep= 'first').sort_values(['Date','new_deaths'], ascending= False)
final_frame_new_deaths.head(5)

,Province/State,Country/Region,Date,total_cases,total_recovered,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
8662,ITALY,ITALY,2020-03-22,59138,7024,5476,5560.0,651.0,0.103774,0.092597,True
12749,SPAIN,SPAIN,2020-03-22,28768,2575,1772,3394.0,397.0,0.133759,0.061596,True
8418,IRAN,IRAN,2020-03-22,21638,7931,1685,1028.0,129.0,0.049879,0.077872,True
6832,FRANCE,FRANCE,2020-03-22,16018,2200,674,1736.0,112.0,0.121552,0.042078,True
14213,UNITED KINGDOM,UNITED KINGDOM,2020-03-22,5683,65,281,665.0,48.0,0.132523,0.049446,True


In [0]:
frame_p2 = final_frame_new_deaths[final_frame_new_deaths['new_deaths'] >1]

pre_plot = prepare_to_plot(locations = frame_p2['Province/State'].head(5).values.tolist(), frame = frame_p2,
                           column_on_x = 'new_deaths', column_on_y = 'Date', 
                           chart_title = "COVID-19 / Number of deaths (day by day) for locations with highest number of new deaths", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Bar')
 
iplot(pre_plot)

In [0]:
# LOCATIONS with highest FATALITY RATE

In [0]:
frame_new.sort_values(['Date', 'FatalityRate'], ascending=False, inplace= True)
frame_new[(frame_new.total_deaths > 1) & (frame_new.new_deaths > 1)].head(5)

,Province/State,Country/Region,Date,total_cases,total_recovered,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
8357,INDONESIA,INDONESIA,2020-03-22,514,29,48,64.0,10.0,0.142222,0.093385,True
8662,ITALY,ITALY,2020-03-22,59138,7024,5476,5560.0,651.0,0.103774,0.092597,True
8479,IRAQ,IRAQ,2020-03-22,233,57,20,19.0,3.0,0.088785,0.085837,True
122,ALGERIA,ALGERIA,2020-03-22,201,65,17,62.0,2.0,0.446043,0.084577,True
8418,IRAN,IRAN,2020-03-22,21638,7931,1685,1028.0,129.0,0.049879,0.077872,True


In [0]:
# LOCATIONS with highest PERCENT OF NEW cases

In [0]:
frame_new.sort_values(['Date', 'PercentOfNew'], ascending=False, inplace= True)
frame_new[(frame_new.total_deaths > 1) & (frame_new.new_deaths > 1)].head(5)

,Province/State,Country/Region,Date,total_cases,total_recovered,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
13603,TURKEY,TURKEY,2020-03-22,1236,0,30,566.0,21.0,0.844776,0.024272,True
1769,BRAZIL,BRAZIL,2020-03-22,1593,2,25,572.0,10.0,0.560235,0.015694,True
6222,ECUADOR,ECUADOR,2020-03-22,789,3,14,283.0,7.0,0.559289,0.017744,True
122,ALGERIA,ALGERIA,2020-03-22,201,65,17,62.0,2.0,0.446043,0.084577,True
8174,HUNGARY,HUNGARY,2020-03-22,131,16,6,28.0,2.0,0.271845,0.045802,True


In [0]:
frame_p3 = final_frame_to_chart[final_frame_to_chart['Province/State'] =="POLAND"]

pre_plot = prepare_to_plot(locations = frame_p3['Province/State'].head(1).values.tolist(), frame = frame_p3,
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / Poland Total Cases day by day", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [0]:
frame_p4 = final_frame_to_chart[final_frame_to_chart['Country/Region'] =="US"]
frame_p4.sort_values(['Date', 'total_cases'], ascending=False, inplace= True)
frame_p4 = frame_p4[frame_p4['Date']> pd.Timestamp(2020, 3,9)]
display(frame_p4.head())
pre_plot = prepare_to_plot(locations = frame_p4['Province/State'].head(5).values.tolist(), frame = frame_p4, 
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / USA  five locations with highest total cases", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

,Province/State,Country/Region,Date,total_cases,total_recovered,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
23485,NEW YORK,US,2020-03-22,15793,0,117,4083.0,57.0,0.348676,0.007408,False
28487,WASHINGTON,US,2020-03-22,1996,0,95,203.0,1.0,0.113218,0.047595,False
23363,NEW JERSEY,US,2020-03-22,1914,0,20,587.0,4.0,0.442351,0.010449,False
15616,CALIFORNIA,US,2020-03-22,1642,0,30,278.0,6.0,0.203812,0.018270,False
19703,ILLINOIS,US,2020-03-22,1049,0,9,296.0,3.0,0.393094,0.008580,False


In [0]:
new = data.groupby(['Date']).sum().reset_index()
new = new[new['Date']> pd.Timestamp(2020, 3, 1)]
new.tail()

,Date,Lat_x,Long_x,total_cases,total_recovered,total_deaths,yesterday_cases,yesterday_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate
56,2020-03-18,14892.9287,-16363.9553,214910,83207,8733,197142.0,7905.0,17768.0,828.0,inf,4.519159
57,2020-03-19,14892.9287,-16363.9553,242708,84854,9867,214910.0,8733.0,27798.0,1134.0,inf,4.247432
58,2020-03-20,14892.9287,-16363.9553,272166,87256,11299,242708.0,9867.0,29458.0,1432.0,inf,4.627253
59,2020-03-21,14892.9287,-16363.9553,304524,91499,12973,272166.0,11299.0,32358.0,1674.0,inf,4.826228
60,2020-03-22,14892.9287,-16363.9553,335955,97704,14632,304524.0,12973.0,31431.0,1659.0,inf,4.819597


In [0]:
dates = new['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=dates, y=new['total_deaths'].values.tolist(), name="total_deaths"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=dates, y=new['total_cases'].values.tolist(), name="total_cases"),
    secondary_y=True,
)
fig.add_trace(
    go.Bar(x=dates, y=new['new_cases'].values.tolist(), name="new_cases"),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=dates, y=new['new_deaths'].values.tolist(), name="new_deaths"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=dates, y=new['total_recovered'].values.tolist(), name="total_recovered"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="World covid statistics"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
#fig.update_yaxes(title_text="<b>deaths - new cases</b>", secondary_y=False)
#fig.update_yaxes(title_text="<b>total cases - total recovered</b>", secondary_y=True)
fig.show()

In [0]:
dates = data['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
frame_pl = data[data['Province/State'] =="POLAND"]
frame_pl = frame_pl[frame_pl['Date']> pd.Timestamp(2020, 3, 1)]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=dates, y=frame_pl['total_deaths'].values.tolist(), name="total_deaths"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=dates, y=frame_pl['total_cases'].values.tolist(), name="total_cases"),
    secondary_y=True,
)
fig.add_trace(
    go.Bar(x=dates, y=frame_pl['new_cases'].values.tolist(), name="new_cases"),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=dates, y=frame_pl['total_recovered'].values.tolist(), name="total_recovered"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="Poland covid statistics"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
fig.update_layout(legend_orientation="v")
# Set y-axes titles
#fig.update_yaxes(title_text="<b>deaths - new cases</b>", secondary_y=False)
#fig.update_yaxes(title_text="<b>total cases - total recovered</b>", secondary_y=True)
fig.show()